<h1 align="center">3.2 Feature Engeneering (Items & Categories)</h1>
<h3 align="center">Dataset: <a href="https://www.kaggle.com/c/competitive-data-science-predict-future-sales">Predict future sales</a></h3>

### Imports

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import category_encoders as ce
import pickle

pd.set_option('display.max_rows', 100)
pd.set_option('display.max_colwidth', None)

### Constants

In [ ]:
ENGLISH = True

DATA_RUS_PATH = "../DATA/1. Original data Russian (96Mb)/"
DATA_ENG_PATH = "../DATA/2. Translated data English (1Mb)/"
DATA_FE_PATH  = "../DATA/3. Feature Engeneering/"
DATA_SUB_PATH = "../DATA/5. Submissions/"

In [ ]:
def save_obj(obj, name):
    with open(DATA_FE_PATH + name + '.pkl', 'wb') as f:
        pickle.dump(obj, f, pickle.HIGHEST_PROTOCOL)

### Load data

In [ ]:
sales = pd.read_csv(DATA_RUS_PATH + "sales_train.csv")          # Dayly sales  Jan 2013 -> Oct 2015
test  = pd.read_csv(DATA_RUS_PATH + "test.csv", index_col="ID") # Predict Noviember 2015
sub   = pd.read_csv(DATA_RUS_PATH + "sample_submission.csv", index_col="ID")

if ENGLISH: 
    shops = pd.read_csv(DATA_ENG_PATH + "shops.csv", index_col="shop_id") # 60
    items = pd.read_csv(DATA_ENG_PATH + "items.csv") # 22170
    cats  = pd.read_csv(DATA_ENG_PATH + "item_categories.csv", index_col="item_category_id") # 84

else:
    shops = pd.read_csv(DATA_RUS_PATH + "shops.csv", index_col="shop_id") # 60
    items = pd.read_csv(DATA_RUS_PATH + "items.csv") # 22170
    cats  = pd.read_csv(DATA_RUS_PATH + "item_categories.csv", index_col="item_category_id") # 84

# <center> Categories (84)

In [ ]:
pd.DataFrame(cats.item_category_name.values.reshape(-1, 4))

,0,1,2,3
0,PC - Headsets / Headphones,Accessories - PS2,Accessories - PS3,Accessories - PS4
1,Accessories - PSP,Accessories - PSVita,Accessories - XBOX 360,Accessories - XBOX ONE
2,Tickets (Digital),Delivery of goods,Game consoles - PS2,Game consoles - PS3
3,Game consoles - PS4,Game consoles - PSP,Game consoles - PSVita,Game consoles - XBOX 360
4,Game consoles - XBOX ONE,Game consoles - Others,Games - PS2,Games - PS3
5,Games - PS4,Games - PSP,Games - PSVita,Games - XBOX 360
6,Games - XBOX ONE,Games - Game accessories,Android games - Number,MAC Games - Number
7,PC Games - Additional Editions,PC Games - Collector's Editions,PC Games - Standard Editions,PC Games - Digital
8,"Payment Cards (Cinema, Music, Games)",Payment Cards - Live!,Payment Cards - Live! (Digital),Payment cards - PSN
9,Payment cards - Windows (Digital),Cinema - Blu-Ray,Cinema - Blu-Ray 3D,Cinema - Blu-Ray 4K


### Get Subcategories

In [ ]:
pd.DataFrame({
    "c0": cats.item_category_name,
    "c1": cats.item_category_name.str.split('-').map(lambda x: x[0].strip()),
    "c2": cats.item_category_name.str.extract(r'(^[\w\s]*)')[0]
})

,c0,c1,c2
item_category_id,,,
0,PC - Headsets / Headphones,PC,PC
1,Accessories - PS2,Accessories,Accessories
2,Accessories - PS3,Accessories,Accessories
3,Accessories - PS4,Accessories,Accessories
4,Accessories - PSP,Accessories,Accessories
5,Accessories - PSVita,Accessories,Accessories
6,Accessories - XBOX 360,Accessories,Accessories
7,Accessories - XBOX ONE,Accessories,Accessories
8,Tickets (Digital),Tickets (Digital),Tickets


In [ ]:
cats['main_category'] = cats['item_category_name'].str.extract(r'(^[\w\s]*)')
cats['main_category'] = cats['main_category'].str.strip()
cats

,item_category_name,main_category
item_category_id,,
0,PC - Headsets / Headphones,PC
1,Accessories - PS2,Accessories
2,Accessories - PS3,Accessories
3,Accessories - PS4,Accessories
4,Accessories - PSP,Accessories
5,Accessories - PSVita,Accessories
6,Accessories - XBOX 360,Accessories
7,Accessories - XBOX ONE,Accessories
8,Tickets (Digital),Tickets


### Encode

In [ ]:
encoder = ce.OrdinalEncoder(cols=["main_category"])

cats  = encoder.fit_transform(cats)
del cats['item_category_name']

cats.head()

/home/javi/.local/lib/python3.9/site-packages/category_encoders/utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):


,main_category
item_category_id,
0,1
1,2
2,2
3,2
4,2


# <center> Items (22170)

In [ ]:
items.head()

,item_id,item_category_id,item_name
0,0,40,!! IN THE POWER OF HAPPINESS (PLAST) D
1,1,76,"! ABBYY FineReader 12 Professional Edition Full [PC, Digital Version]"
2,2,40,*** IN THE GLORY OF THE GLORY (UNV) D
3,3,40,*** BLUE WAVE (Univ) D
4,4,40,*** BOX (GLASS) D


### Clean

In [ ]:
items['item_name'] = items['item_name'].str.lower()
items['item_name'] = items['item_name'].str.replace('.', '')
for i in [r'[^\w\d\s\.]', r'\bthe\b', r'\bin\b', r'\bis\b',
          r'\bfor\b', r'\bof\b', r'\bon\b', r'\band\b',  
          r'\bto\b', r'\bwith\b' , r'\byo\b']:
    items['item_name'] = items['item_name'].str.replace(i, ' ')
items['item_name'] = items['item_name'].str.replace(r'\b.\b', ' ')

items.head()

<ipython-input-10-0b8bc6632c2f>:2: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will*not* be treated as literal strings when regex=True.
  items['item_name'] = items['item_name'].str.replace('.', '')
<ipython-input-10-0b8bc6632c2f>:6: FutureWarning: The default value of regex will change from True to False in a future version.
  items['item_name'] = items['item_name'].str.replace(i, ' ')
<ipython-input-10-0b8bc6632c2f>:7: FutureWarning: The default value of regex will change from True to False in a future version.
  items['item_name'] = items['item_name'].str.replace(r'\b.\b', ' ')


,item_id,item_category_id,item_name
0,0,40,power happiness plast
1,1,76,abbyy finereader 12 professional edition full pc digital version
2,2,40,glory glory unv
3,3,40,blue wave univ
4,4,40,box glass


### Find duplicates

In [ ]:
dupes = items[(items.duplicated(subset=['item_name','item_category_id'],keep=False))]
dupes['in_test'] = dupes.item_id.isin(test.item_id.unique())
dupes = dupes.groupby('item_name').agg({'item_id':["count",'first','last'],'in_test':['first','last']})
dupes.head(15)

<ipython-input-11-62244aea463d>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dupes['in_test'] = dupes.item_id.isin(test.item_id.unique())


item_id              in_test  \
                                                count first   last   first   
item_name                                                                    
                                                    2    32  10725    True   
      all   ballads     80                          2  7507   7508    True   
      art   electro swing     digipack              2  7453   7454   False   
      best   lounge     4cd  digibook               2  7456   7457   False   
      instrumental hits best                        2  7516   7517   False   
      love vocal house     2cd  digipack            2  7368   7369   False   
    armada lounge     2cd  digipack                 2  7258   7259   False   
    best radio tracks     2cd  digipack             3  7272   7275   False   
    classic trance tracks     3cd  digibook         2  7296   7297   False   
    clubber   mood   ibiza     2cd  digipack        3  7306   7308   False   
    dance   floor hits     mp3 cd   jewel           3  7311   7313   False   
    dance top hits     4cd  digibook                5  7319   7323   False   
    deep breeze     2cd  digipack                   2  7325   7326   False   
    deep house  dreams     2cd  digipack            3  7329   7331   False   
    deep house  series     2cd  digipack            2  7335   7338   False   

                                                      
                                                last  
item_name                                             
                                               False  
      all   ballads     80                      True  
      art   electro swing     digipack         False  
      best   lounge     4cd  digibook          False  
      instrumental hits best                   False  
      love vocal house     2cd  digipack       False  
    armada lounge     2cd  digipack            False  
    best radio tracks     2cd  digipack        False  
    classic trance tracks     3cd  digibook    False  
    clubber   mood   ibiza     2cd  digipack    True  
    dance   floor hits     mp3 cd   jewel      False  
    dance top hits     4cd  digibook           False  
    deep breeze     2cd  digipack              False  
    deep house  dreams     2cd  digipack        True  
    deep house  series     2cd  digipack        True

### Remove duplicated

In [ ]:
# A) if both item id's are in the test set do nothing
dupes = dupes[(dupes[('in_test', 'first')]==False) | (dupes[('in_test', 'last')]==False)]

# B) if only the first id is in the test set assign this id to both
temp = dupes[dupes[('in_test', 'first')]==True]
keep_first = dict(zip(temp[('item_id', 'last')], temp[('item_id',  'first')]))

# C) if neither id or only the second id is in the test set, assign the second id to both
temp = dupes[dupes[('in_test', 'first')]==False]
keep_second = dict(zip(temp[('item_id', 'first')], temp[('item_id',  'last')]))
item_map = {**keep_first, **keep_second}

item_map

{10725: 32,
 17045: 17042,
 21012: 21010,
 21088: 21086,
 21115: 21113,
 21125: 21123,
 21138: 21136,
 169: 168,
 573: 571,
 11200: 11199,
 8358: 8357,
 16948: 16946,
 1853: 1829,
 9744: 9740,
 8857: 8855,
 21598: 21597,
 10946: 10945,
 18946: 18945,
 8526: 8524,
 22038: 22035,
 18618: 18617,
 10373: 10372,
 9802: 9767,
 11689: 11688,
 20100: 20099,
 22112: 22111,
 12379: 12378,
 4194: 4178,
 4195: 4179,
 4221: 4217,
 13028: 13027,
 16628: 13804,
 14104: 14103,
 16550: 16549,
 5354: 5353,
 15597: 15595,
 17018: 17017,
 5947: 5945,
 17922: 17921,
 18413: 18411,
 8248: 8247,
 21070: 21069,
 21209: 21207,
 21909: 21906,
 18834: 18831,
 19054: 19053,
 18725: 18724,
 20020: 20019,
 15708: 15698,
 7453: 7454,
 7456: 7457,
 7516: 7517,
 7368: 7369,
 7258: 7259,
 7272: 7275,
 7296: 7297,
 7306: 7308,
 7311: 7313,
 7319: 7323,
 7325: 7326,
 7329: 7331,
 7335: 7338,
 7333: 7334,
 7341: 7342,
 21239: 21240,
 21242: 21243,
 21245: 21246,
 21248: 21250,
 7352: 7353,
 7357: 7358,
 7364: 7366,
 7385:

### Save item map

In [ ]:
save_obj(item_map, "duplicated_items_replacement")

### Extract first n characters of name

In [ ]:
items['item_name_no_space'] = items['item_name'].str.replace(' ', '')
items['item_name_first4']   = [x[:4] for x in items['item_name_no_space']]
items['item_name_first6']   = [x[:6] for x in items['item_name_no_space']]
items['item_name_first11']  = [x[:11] for x in items['item_name_no_space']]
del items['item_name_no_space']

items.head()

,item_id,item_category_id,item_name,item_name_first4,item_name_first6,item_name_first11
0,0,40,power happiness plast,powe,powerh,powerhappin
1,1,76,abbyy finereader 12 professional edition full pc digital version,abby,abbyyf,abbyyfinere
2,2,40,glory glory unv,glor,gloryg,glorygloryu
3,3,40,blue wave univ,blue,bluewa,bluewaveuni
4,4,40,box glass,boxg,boxgla,boxglass


### Encode

In [ ]:
encoder = ce.OrdinalEncoder(cols=["item_name_first4", "item_name_first6", "item_name_first11"])

items  = encoder.fit_transform(items)
del items['item_name']

items.head()

/home/javi/.local/lib/python3.9/site-packages/category_encoders/utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):


,item_id,item_category_id,item_name_first4,item_name_first6,item_name_first11
0,0,40,1,1,1
1,1,76,2,2,2
2,2,40,3,3,3
3,3,40,4,4,4
4,4,40,5,5,5


### JOIN with cats

In [ ]:
items = pd.merge(items, cats, on="item_category_id", how="left")
items.head()

,item_id,item_category_id,item_name_first4,item_name_first6,item_name_first11,main_category
0,0,40,1,1,1,12
1,1,76,2,2,2,16
2,2,40,3,3,3,12
3,3,40,4,4,4,12
4,4,40,5,5,5,12


### Save

In [ ]:
items.to_csv(DATA_FE_PATH + "items.csv")

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=915d6e57-4e7f-4527-840e-d54ca0b4f460' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>